# 3rd Level Model Structure: Initialize Procedure (20+ stages)

In [1]:
# system imports
import sys
import os
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../..'))

import numpy as np
from matplotlib import pyplot as plt

import ipywidgets as widgets
from IPython.display import display, clear_output

import pickle
from copy import deepcopy

In [2]:
# pyomo imports
from pyomo import environ as pe
from global_sets.component import m

from stages.reactive_stage import reactive_stage_rule
from stages.condenser_stage import condenser_stage_rule
from stages.reboiler_stage import reboiler_stage_rule

from utility.display_utility import trans_product_mole, trans_product_mass, beautify, beautify_reactive
from utility.model_utility import add_dual, update_dual, delete_dual, check_DOF, check_violate_constraint
from utility.data_utility import cal_cnumber

model = pe.ConcreteModel(name='reactive_distillation')

# Global Set

In [3]:
model.TRAY = pe.RangeSet(1,10)

# Construct reactive stages and Initialization separately

In [4]:
model.reactive = pe.Block(model.TRAY,rule=reactive_stage_rule)
add_dual(pe,model)

> Importing Reactive Stage......
> Adding the following local variable:
------------------------------------
| reactive[1].x_
| reactive[1].y_
| reactive[1].x
| reactive[1].y
| reactive[1].z
| reactive[1].L
| reactive[1].V
| reactive[1].F
| reactive[1].H_L_
| reactive[1].H_V_
| reactive[1].H_L
| reactive[1].H_V
| reactive[1].H_F
| reactive[1].T
| reactive[1].T_F
| reactive[1].P
| reactive[1].f_V
| reactive[1].f_L
| reactive[1].cat
| reactive[1].Q_main
| reactive[1].r_total_comp
| reactive[1].PR_L
| reactive[1].PR_V
------------------------------------

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[1].kinetics_block.k_FT
| reactive[1].kinetics_block.r_FT_total
| reactive[1].kinetics_block.g0_FT
| reactive[1].kinetics_block.alpha
| reactive[1].kinetics_block.r_FT_cnum
| reactive[1].kinetics_block.r_FT_comp
| reactive[1].kinetics_block.k_WGS
| reactive[1].kinetics_block.Ke_WGS
| reactive[1].kinetics_b

> Importing MPCC_P_pf Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[4].MPCC.s_L
| reactive[4].MPCC.s_V
| reactive[4].MPCC.pf
| reactive[4].MPCC.rho
--------------------------------------------------
> Spliting pressure used in VLE

> Importing Reactive Stage......
> Adding the following local variable:
------------------------------------
| reactive[5].x_
| reactive[5].y_
| reactive[5].x
| reactive[5].y
| reactive[5].z
| reactive[5].L
| reactive[5].V
| reactive[5].F
| reactive[5].H_L_
| reactive[5].H_V_
| reactive[5].H_L
| reactive[5].H_V
| reactive[5].H_F
| reactive[5].T
| reactive[5].T_F
| reactive[5].P
| reactive[5].f_V
| reactive[5].f_L
| reactive[5].cat
| reactive[5].Q_main
| reactive[5].r_total_comp
| reactive[5].PR_L
| reactive[5].PR_V
------------------------------------

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[5].kin

| reactive[9].r_total_comp
| reactive[9].PR_L
| reactive[9].PR_V
------------------------------------

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[9].kinetics_block.k_FT
| reactive[9].kinetics_block.r_FT_total
| reactive[9].kinetics_block.g0_FT
| reactive[9].kinetics_block.alpha
| reactive[9].kinetics_block.r_FT_cnum
| reactive[9].kinetics_block.r_FT_comp
| reactive[9].kinetics_block.k_WGS
| reactive[9].kinetics_block.Ke_WGS
| reactive[9].kinetics_block.r_WGS
| reactive[9].kinetics_block.r_WGS_comp
--------------------------------------------------

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[9].energy_block.dH_F
| reactive[9].energy_block.dH_V
| reactive[9].energy_block.dH_L
| reactive[9].energy_block.dH_vap
--------------------------------------------------

> Importing VLE Blocks......
> Adding the followi

In [5]:
# in/out variable
for j in model.reactive:
    model.reactive[j].x_.fix(0)
    model.reactive[j].y_.fix(0)
    model.reactive[j].L['in'].fix(0)
    model.reactive[j].V['in'].fix(0)
    model.reactive[j].H_L_.fix(0)
    model.reactive[j].H_V_.fix(0)

# operating parameters
for j in model.reactive:
    model.reactive[j].cat.fix(3000)
    model.reactive[j].P.fix(20)
    model.reactive[j].VLE_block.n_ave.fix(20)
    
    model.reactive[j].F.fix(1)
    model.reactive[j].T_F.fix(200+273.15)
    model.reactive[j].z['CO'].fix(1/(1+2)-0/2)
    model.reactive[j].z['H2'].fix(2/(1+2)-0/2)
    model.reactive[j].z['C30H62'].fix(0)
    
    model.reactive[j].PR_L.fix(1)
    model.reactive[j].PR_V.fix(1)
    
    # model.reactive[j].Q_main.fix(0)
    model.reactive[j].T.setub(220+273.15)
    model.reactive[j].T.setlb(200+273.15)

In [6]:
check_DOF(pe,model)

Active Equality Constraints:	 11840
Active Inequality Constraints:	 10
Active Variables:		 13600
Fixed Variables:		 1720
DOF:				 40


In [7]:
model.obj = pe.Objective(expr = sum(model.reactive[j].T - model.reactive[j].MPCC.pf for j in model.reactive),sense=pe.maximize)

In [8]:
opt = pe.SolverFactory('ipopt')

opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma86'

opt.options['linear_system_scaling '] = 'mc19'
opt.options['linear_scaling_on_demand '] = 'no'

# opt.options['warm_start_init_point'] = 'yes'
# opt.options['warm_start_bound_push'] = 1e-20
# opt.options['warm_start_mult_bound_push'] = 1e-20
# opt.options['mu_init'] = 1e-6

# opt.options['halt_on_ampl_error'] = 'yes'

opt.options['max_iter'] = 7000

In [9]:
%%time

results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling =mc19
linear_scaling_on_demand =no
max_iter=7000


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with

CPU times: user 1.52 s, sys: 89.2 ms, total: 1.61 s
Wall time: 12.6 s


In [10]:
beautify_reactive(pe,model)

------------------------------------------------------------------------------------------------------------
stages		T		Q		V_out		L_out		L_P		P_VLE
Reactive[1]	220.00		-43.29		 0.0000		0.00000		0.00642		20.00000
Reactive[2]	220.00		-43.29		 0.0000		0.00000		0.00642		20.00000
Reactive[3]	220.00		-43.29		 0.0000		0.00000		0.00642		20.00000
Reactive[4]	220.00		-43.29		 0.0000		0.00000		0.00642		20.00000
Reactive[5]	220.00		-43.29		 0.0000		0.00000		0.00642		20.00000
Reactive[6]	220.00		-43.29		 0.0000		0.00000		0.00642		20.00000
Reactive[7]	220.00		-43.29		 0.0000		0.00000		0.00642		20.00000
Reactive[8]	220.00		-43.29		 0.0000		0.00000		0.00642		20.00000
Reactive[9]	220.00		-43.29		 0.0000		0.00000		0.00642		20.00000
Reactive[10]	220.00		-43.29		 0.0000		0.00000		0.00642		20.00000


# Connect reactive stages and Initialize connectively

### Vapor Between Reactive Stages

In [11]:
def V_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].V['in'] == model.reactive[j+1].V['out']
model.V_between_con = pe.Constraint(model.TRAY,rule=V_between_rule)

def Vy_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].y_['in',i] == model.reactive[j+1].y[i]
model.Vy_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Vy_between_rule)

def Vh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].H_V_['in'] == model.reactive[j+1].H_V
model.Vh_between_con = pe.Constraint(model.TRAY,rule=Vh_between_rule)

### Liquid Between Reactive Stages

In [12]:
def L_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].L['in'] == model.reactive[j].L['out']
model.L_between_con = pe.Constraint(model.TRAY,rule=L_between_rule)

def Lx_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].x_['in',i] == model.reactive[j].x[i]
model.Ly_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Lx_between_rule)

def Lh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].H_L_['in'] == model.reactive[j].H_L
model.Lh_between_con = pe.Constraint(model.TRAY,rule=Lh_between_rule)

### Unlock liquid and vapor in-flow

In [13]:
for j in model.reactive:
    if j != model.TRAY.first():
        model.reactive[j].x_.unfix()
        model.reactive[j].H_L_.unfix()
        model.reactive[j].L['in'].unfix()
    if j != model.TRAY.last():
        model.reactive[j].y_.unfix()
        model.reactive[j].V['in'].unfix()
        model.reactive[j].H_V_.unfix()

In [14]:
check_DOF(pe,model)

Active Equality Constraints:	 13298
Active Inequality Constraints:	 10
Active Variables:		 13600
Fixed Variables:		 262
DOF:				 40


In [15]:
opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6

results = opt.solve(model,tee=False)
update_dual(pe,model)

### Iteratively reduce stage product draw ratio

In [16]:
%%time

PR_range = np.linspace(1,0,11)
for r in PR_range:
    for j in model.reactive:
        model.reactive[j].PR_L.fix(r)
        model.reactive[j].PR_V.fix(r)
        
    results = opt.solve(model,tee=False)
    update_dual(pe,model)
    print('\nWorking on PR ratio = {:.2f}'.format(r))
    beautify_reactive(pe,model)


Working on PR ratio = 1.00
------------------------------------------------------------------------------------------------------------
stages		T		Q		V_out		L_out		L_P		P_VLE
Reactive[1]	220.00		-43.29		 0.0000		0.00000		0.00642		20.00000
Reactive[2]	220.00		-43.29		 0.0000		0.00000		0.00642		20.00000
Reactive[3]	220.00		-43.29		 0.0000		0.00000		0.00642		20.00000
Reactive[4]	220.00		-43.29		 0.0000		0.00000		0.00642		20.00000
Reactive[5]	220.00		-43.29		 0.0000		0.00000		0.00642		20.00000
Reactive[6]	220.00		-43.29		 0.0000		0.00000		0.00642		20.00000
Reactive[7]	220.00		-43.29		 0.0000		0.00000		0.00642		20.00000
Reactive[8]	220.00		-43.29		 0.0000		0.00000		0.00642		20.00000
Reactive[9]	220.00		-43.29		 0.0000		0.00000		0.00642		20.00000
Reactive[10]	220.00		-43.29		 0.0000		0.00000		0.00642		20.00000

Working on PR ratio = 0.90
------------------------------------------------------------------------------------------------------------
stages		T		Q		V_out		L_out		L_P		P_VLE
Reactiv

# Construct a single condenser and Initialize

### Deactivate the reactive part first

In [17]:
for i in model.block_data_objects():
    if i.name != 'reactive_distillation':
        i.deactivate()
for i in model.component_objects(pe.Constraint, active=True):
    i.deactivate()
check_DOF(pe,model)

Active Equality Constraints:	 0
Active Inequality Constraints:	 0
Active Variables:		 0
Fixed Variables:		 0
DOF:				 0


In [18]:
model.condenser = pe.Block(rule=condenser_stage_rule)

| Importing Condenser Stage......
| Adding the following local variable:
------------------------------------
| condenser.T
| condenser.T_F
| condenser.P
| condenser.Q_main
| condenser.x_
| condenser.y_
| condenser.x
| condenser.y
| condenser.z
| condenser.L
| condenser.W
| condenser.V
| condenser.F
| condenser.H_L_
| condenser.H_V_
| condenser.H_L
| condenser.H_V
| condenser.H_F
| condenser.f_V
| condenser.f_L
| condenser.PR_L
------------------------------------

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.energy_block.dH_F
| condenser.energy_block.dH_V
| condenser.energy_block.dH_L
| condenser.energy_block.dH_vap
--------------------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.VLE_block.n_ave
| condenser.VLE_block.n_ave_cal
| condenser.VLE_block.Hen
| condenser.VLE_block.Hen0
| 

### Use the reactive sections vapor output as input to condenser

In [19]:
# in/out variables
model.condenser.x_.fix(0)
for i in m.COMP_TOTAL:
    model.condenser.y_['in',i].fix(model.reactive[model.TRAY.first()].y[i].value)
model.condenser.V['in'].fix(model.reactive[model.TRAY.first()].V['out'].value)
model.condenser.L['in'].fix(0)
model.condenser.V['P'].fix(0)
model.condenser.H_L_.fix(0)
model.condenser.H_V_.fix(model.reactive[model.TRAY.first()].H_V.value)

# operating parameters
model.condenser.P.fix(19)
model.condenser.T_F.fix(200+273.15)
model.condenser.F.fix(0)
model.condenser.z.fix(0)
model.condenser.VLE_block.n_ave.fix(4)
model.condenser.PR_L.fix(1)

model.condenser.T.setub(30+273.15)

In [20]:
model.del_component(model.obj)
model.obj = pe.Objective(expr = model.condenser.T, sense = pe.maximize)

delete_dual(pe,model)
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


In [21]:
check_DOF(pe,model)

Active Equality Constraints:	 958
Active Inequality Constraints:	 0
Active Variables:		 1130
Fixed Variables:		 171
DOF:				 1


In [22]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling =mc19
linear_scaling_on_demand =no
max_iter=7000
warm_start_init_point=yes
warm_start_bound_push=1e-20
warm_start_mult_bound_push=1e-20
mu_init=1e-06


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 7000                  yes
                                 mu_init = 1e-06                 yes
                      print_user_options = yes                   yes
                   warm_start_bound_push = 1e-20                 yes
                   warm_start_init_point = yes                   yes
              warm_start_mult_bound_push = 1e-20                 yes

****************************************************

 102 -3.0315000e+02 2.09e+00 3.38e+02  -6.0 2.21e+00    -  8.46e-01 5.26e-02h  1
 103 -3.0315000e+02 5.93e-03 3.09e+01  -6.0 2.09e+00    -  9.06e-01 1.00e+00h  1
 104 -3.0315000e+02 1.18e-05 1.35e-04  -6.0 8.51e-03    -  1.00e+00 1.00e+00h  1
 105 -3.0315000e+02 4.97e-11 6.77e-11  -6.0 1.46e-05    -  1.00e+00 1.00e+00h  1
 106 -3.0315000e+02 1.50e-10 9.05e-07  -9.0 3.94e-05    -  1.00e+00 1.00e+00h  1
 107 -3.0315000e+02 9.94e-11 1.00e-14  -9.0 4.16e-10    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 107

                                   (scaled)                 (unscaled)
Objective...............:  -3.0315000303049999e+02   -3.0315000303049999e+02
Dual infeasibility......:   1.0019762797242038e-14    1.0019762797242038e-14
Constraint violation....:   9.9373484809851809e-11    9.9373484809851809e-11
Complementarity.........:   1.0000000000000491e-09    1.0000000000000491e-09
Overall NLP error.......:   1.0000000000000491e-09    1.0000000000000491e-09


Number of objective func

# Construct a single reboiler and Initialize

### Deactivate the condenser part first

In [23]:
model.condenser.deactivate()
check_DOF(pe,model)

Active Equality Constraints:	 0
Active Inequality Constraints:	 0
Active Variables:		 0
Fixed Variables:		 0
DOF:				 0


In [24]:
model.reboiler = pe.Block(rule=reboiler_stage_rule)

> Importing Non Reactive Stage......
> Adding the following local variable:
------------------------------------
| reboiler.T_F
| reboiler.P
| reboiler.Q_main
| reboiler.x_
| reboiler.y_
| reboiler.x
| reboiler.y
| reboiler.z
| reboiler.L
| reboiler.V
| reboiler.F
| reboiler.H_L_
| reboiler.H_V_
| reboiler.H_L
| reboiler.H_V
| reboiler.T
| reboiler.H_F
| reboiler.f_V
| reboiler.f_L
------------------------------------

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| reboiler.energy_block.dH_F
| reboiler.energy_block.dH_V
| reboiler.energy_block.dH_L
| reboiler.energy_block.dH_vap
--------------------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| reboiler.VLE_block.P_VLE
| reboiler.VLE_block.n_ave
| reboiler.VLE_block.n_ave_cal
| reboiler.VLE_block.Hen
| reboiler.VLE_block.Hen0
| reboiler.VLE_block.gamma
| r

### Use the reactive sections liquid output as input to reboiler

In [25]:
# in/out variables
model.reboiler.y_.fix(0)
for i in m.COMP_TOTAL:
    model.reboiler.x_['in',i].fix(model.reactive[model.TRAY.last()].x[i].value)
model.reboiler.L['in'].fix(model.reactive[model.TRAY.last()].L['out'].value)
model.reboiler.V['in'].fix(0)
model.reboiler.L['P'].fix(0)
model.reboiler.V['P'].fix(0)
model.reboiler.H_L_.fix(model.reactive[model.TRAY.last()].H_L.value)
model.reboiler.H_V_.fix(0)

# operating parameters
model.reboiler.P.fix(20)
model.reboiler.T_F.fix(200+273.15)
model.reboiler.F.fix(0)
model.reboiler.z.fix(0)
model.reboiler.VLE_block.n_ave.fix(20)

model.reboiler.T.setub(model.reactive[model.TRAY.last()].T.value)

In [26]:
model.del_component(model.obj)
model.obj = pe.Objective(expr = model.reboiler.T - model.reboiler.MPCC.pf, sense = pe.maximize)

delete_dual(pe,model)
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


In [27]:
check_DOF(pe,model)

Active Equality Constraints:	 957
Active Inequality Constraints:	 1
Active Variables:		 1132
Fixed Variables:		 171
DOF:				 4


In [28]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling =mc19
linear_scaling_on_demand =no
max_iter=7000
warm_start_init_point=yes
warm_start_bound_push=1e-20
warm_start_mult_bound_push=1e-20
mu_init=1e-06


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 7000                  yes
                                 mu_init = 1e-06                 yes
                      print_user_options = yes                   yes
                   warm_start_bound_push = 1e-20                 yes
                   warm_start_init_point = yes                   yes
              warm_start_mult_bound_push = 1e-20                 yes

****************************************************

Constraint violation....:   1.4297896200332616e-10    1.4297896200332616e-10
Complementarity.........:   1.0013787684185202e-09    1.0013787684185202e-09
Overall NLP error.......:   7.0982461115517981e-09    7.0982461115517981e-09


Number of objective function evaluations             = 81
Number of objective gradient evaluations             = 44
Number of equality constraint evaluations            = 81
Number of inequality constraint evaluations          = 82
Number of equality constraint Jacobian evaluations   = 72
Number of inequality constraint Jacobian evaluations = 72
Number of Lagrangian Hessian evaluations             = 67
Total CPU secs in IPOPT (w/o function evaluations)   =      2.594
Total CPU secs in NLP function evaluations           =      0.133

EXIT: Optimal Solution Found.


# Initialization is complete

In [29]:
beautify(pe,model)

Here comes the result:
------------------------------------------------------------------------------------------------------------
stages		T		Q		V_out		L_out		L_P		W
Condenser	30.00		-134.95		2.54837		0.00000		0.13822		2.08954

stages		T		Q		V_out		L_out		L_P		P_VLE
Reactive[1]	220.00		-43.29		4.77612		0.00642		0.00000		20.00000
Reactive[2]	220.00		-43.29		4.29851		0.01285		0.00000		20.00000
Reactive[3]	220.00		-43.29		3.82090		0.01927		0.00000		20.00000
Reactive[4]	220.00		-43.29		3.34329		0.02570		0.00000		20.00000
Reactive[5]	220.00		-43.29		2.86567		0.03212		0.00000		20.00000
Reactive[6]	220.00		-43.29		2.38806		0.03854		0.00000		20.00000
Reactive[7]	220.00		-43.29		1.91045		0.04497		0.00000		20.00000
Reactive[8]	220.00		-43.29		1.43284		0.05139		0.00000		20.00000
Reactive[9]	220.00		-43.29		0.95522		0.05781		0.00000		20.00000
Reactive[10]	220.00		-43.29		0.47761		0.06424		0.00000		20.00000
Reboiler	220.00		0.00		0.00000		0.06424				19.99770
---------------------------------------

# Linking Condenser and Reboiler

### Activate previously disabled constraint and variables

In [30]:
for i in model.block_data_objects():
    if i.name != 'reactive_distillation':
        i.activate()
for i in model.component_objects(pe.Constraint):
    i.activate()

### Condenser Vapor

In [31]:
def V_condenser_rule(model):
    return model.reactive[model.TRAY.first()].V['out'] == model.condenser.V['in']
model.V_condenser_con = pe.Constraint(rule=V_condenser_rule)

def Vy_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].y[i] == model.condenser.y_['in',i]
model.Vy_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Vy_condenser_rule)

def Vh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_V == model.condenser.H_V_['in']
model.Vh_condenser_con = pe.Constraint(rule=Vh_condenser_rule)

### Condenser Liquid

In [32]:
def L_condenser_rule(model):
    return model.reactive[model.TRAY.first()].L['in'] == model.condenser.L['out']
model.L_condenser_con = pe.Constraint(rule=L_condenser_rule)

def Lx_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].x_['in',i] == model.condenser.x[i]
model.Lx_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Lx_condenser_rule)

def Lh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_L_['in'] == model.condenser.H_L
model.Lh_condenser_con = pe.Constraint(rule=Lh_condenser_rule)

### Reboiler Vapor

In [33]:
def V_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].V['in'] == model.reboiler.V['out']
model.V_reboiler_con = pe.Constraint(rule=V_reboiler_rule)

def Vy_reboiler_rule(model,i):
    return model.reactive[model.TRAY.last()].y_['in',i] == model.reboiler.y[i]
model.Vy_reboiler_con = pe.Constraint(m.COMP_TOTAL,rule=Vy_reboiler_rule)

def Vh_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].H_V_['in'] == model.reboiler.H_V
model.Vh_reboiler_con = pe.Constraint(rule=Vh_reboiler_rule)

### Reboiler Liquid

In [34]:
def L_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].L['out'] + 1e-6 == model.reboiler.L['in']
model.L_reboiler_con = pe.Constraint(rule=L_reboiler_rule)

def Lx_reboiler_rule(model,i):
    return model.reactive[model.TRAY.last()].x[i] == model.reboiler.x_['in',i]
model.Lx_reboiler_con = pe.Constraint(m.COMP_TOTAL,rule=Lx_reboiler_rule)

def Lh_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].H_L == model.reboiler.H_L_['in']
model.Lh_reboiler_con = pe.Constraint(rule=Lh_reboiler_rule)

In [35]:
model.del_component(model.obj)
model.obj = pe.Objective(expr = sum(model.reactive[j].T - model.reactive[j].MPCC.pf for j in model.reactive)\
                         - model.reboiler.MPCC.pf ,sense=pe.maximize)

In [36]:
delete_dual(pe,model)
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


# Fixing Redundent Stream Variables

### Condenser

In [37]:
# in/out variables
model.condenser.x_.fix(0)
for i in m.COMP_TOTAL:
    model.condenser.y_['in',i].unfix()
model.condenser.V['in'].unfix()
model.condenser.L['in'].fix(0)
model.condenser.V['P'].fix(0)
model.condenser.H_L_.fix(0)
model.condenser.H_V_.unfix()

# operating parameters
model.condenser.P.fix(19)
model.condenser.T_F.fix(200+273.15)
model.condenser.F.fix(0)
model.condenser.z.fix(0)
model.condenser.VLE_block.n_ave.fix(4)
model.condenser.PR_L.fix(1)

model.condenser.T.fix(30+273.15)

### Reboiler

In [38]:
# in/out variables
model.reboiler.y_.fix(0)
for i in m.COMP_TOTAL:
    model.reboiler.x_['in',i].unfix()
model.reboiler.L['in'].unfix()
model.reboiler.V['in'].fix(0)
model.reboiler.L['P'].fix(0)
model.reboiler.V['P'].fix(0)
model.reboiler.H_L_.unfix()
model.reboiler.H_V_.fix(0)

# operating parameters
model.reboiler.P.fix(20)
model.reboiler.T_F.fix(200+273.15)
model.reboiler.F.fix(0)
model.reboiler.z.fix(0)
model.reboiler.VLE_block.n_ave.fix(20)

model.reboiler.T.fix(model.reactive[model.TRAY.last()].T.value)

### Reactive

In [39]:
# unlock reflux and reboiler vapor
for j in model.reactive:
    model.reactive[j].x_.unfix()
    model.reactive[j].H_L_.unfix()
    model.reactive[j].L['in'].unfix()
    model.reactive[j].y_.unfix()
    model.reactive[j].V['in'].unfix()
    model.reactive[j].H_V_.unfix()

for j in model.reactive:
    model.reactive[j].cat.fix(3000)
    model.reactive[j].P.fix(20)
    model.reactive[j].VLE_block.n_ave.fix(20)
    
    model.reactive[j].F.fix(1)
    model.reactive[j].T_F.fix(200+273.15)
    model.reactive[j].z['CO'].fix(1/(1+2)-0/2)
    model.reactive[j].z['H2'].fix(2/(1+2)-0/2)
    model.reactive[j].z['C30H62'].fix(0)
    
    model.reactive[j].PR_L.fix(0)
    model.reactive[j].PR_V.fix(0)
    
    # model.reactive[j].Q_main.fix(0)
    model.reactive[j].T.setub(220+273.15)
    model.reactive[j].T.setlb(200+273.15)

# Connected Solve

In [40]:
check_DOF(pe,model)

Active Equality Constraints:	 15537
Active Inequality Constraints:	 11
Active Variables:		 15862
Fixed Variables:		 282
DOF:				 43


In [41]:
%%time

results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling =mc19
linear_scaling_on_demand =no
max_iter=7000
warm_start_init_point=yes
warm_start_bound_push=1e-20
warm_start_mult_bound_push=1e-20
mu_init=1e-06


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 7000                  yes
                                 mu_init = 1e-06                 yes
                      print_user_options = yes                   yes
                   warm_start_bound_push = 1e-20                 yes
                   warm_start_init_point = yes                   yes
              warm_start_mult_bound_push = 1e-20                 yes

****************************************************

In [42]:
beautify(pe,model)

Here comes the result:
------------------------------------------------------------------------------------------------------------
stages		T		Q		V_out		L_out		L_P		W
Condenser	30.00		-134.95		2.54837		0.00000		0.13822		2.08954

stages		T		Q		V_out		L_out		L_P		P_VLE
Reactive[1]	220.00		-43.29		4.77612		0.00642		0.00000		20.00000
Reactive[2]	220.00		-43.29		4.29851		0.01285		0.00000		20.00000
Reactive[3]	220.00		-43.29		3.82090		0.01927		0.00000		20.00000
Reactive[4]	220.00		-43.29		3.34329		0.02570		0.00000		20.00000
Reactive[5]	220.00		-43.29		2.86567		0.03212		0.00000		20.00000
Reactive[6]	220.00		-43.29		2.38806		0.03854		0.00000		20.00000
Reactive[7]	220.00		-43.29		1.91045		0.04497		0.00000		20.00000
Reactive[8]	220.00		-43.29		1.43284		0.05139		0.00000		20.00000
Reactive[9]	220.00		-43.29		0.95523		0.05781		0.00000		20.00000
Reactive[10]	220.00		-43.29		0.47761		0.06424		0.00000		20.00000
Reboiler	220.00		0.00		0.00000		0.06424				19.99724
---------------------------------------

In [43]:
def beautify_reactive(pyomo,model):
    print('-'*108)
    print('stages\t\tT\t\tQ\t\tV_out\t\tL_out\t\tL_P\t\tP_VLE')
    for j in model.reactive:
        temp_num = model.reactive[j].T.value - 273.15, model.reactive[j].Q_main.value,model.reactive[j].V['out'].value,\
            model.reactive[j].L['out'].value,model.reactive[j].L['P'].value,model.reactive[j].VLE_block.P_VLE.value
        temp_string = ['{:.5f}'.format(i) for i in temp_num]
        print('Reactive[{}]\t{:.5s}\t\t{:.5s}\t\t{:.6s}\t\t{:.6s}\t\t{:.6s}\t\t{:.6s}'.format(j,*temp_string))

In [44]:
beautify_reactive(pe,model)

------------------------------------------------------------------------------------------------------------
stages		T		Q		V_out		L_out		L_P		P_VLE
Reactive[1]	220.0		-43.2		4.7761		0.0064		0.0000		20.000
Reactive[2]	220.0		-43.2		4.2985		0.0128		0.0000		20.000
Reactive[3]	220.0		-43.2		3.8209		0.0192		0.0000		20.000
Reactive[4]	220.0		-43.2		3.3432		0.0257		0.0000		20.000
Reactive[5]	220.0		-43.2		2.8656		0.0321		0.0000		20.000
Reactive[6]	220.0		-43.2		2.3880		0.0385		0.0000		20.000
Reactive[7]	220.0		-43.2		1.9104		0.0449		0.0000		20.000
Reactive[8]	220.0		-43.2		1.4328		0.0513		0.0000		20.000
Reactive[9]	220.0		-43.2		0.9552		0.0578		0.0000		20.000
Reactive[10]	220.0		-43.2		0.4776		0.0642		0.0000		20.000


In [45]:
model.reactive[1].VLE_block.P_sat_Y.pprint()

P_sat_Y : Size=70, Index=reactive[1].VLE_block.COMP_NONHENRY
    Key     : Lower               : Value               : Upper                : Fixed : Stale : Domain
     C10H20 : -2.4614099080125302 :  1.1263884445865826 :    3.474028484362643 : False : False :  Reals
     C10H22 : -2.5955361701142494 :  1.0500355364861729 :   3.4128464550154303 : False : False :  Reals
     C11H22 : -3.3207367315668512 :  0.6408656964256351 :    3.089238578836299 : False : False :  Reals
     C11H24 :  -3.453088045682415 :  0.5668133923450308 :   3.0314030121743656 : False : False :  Reals
     C12H24 :  -4.169583339226395 :  0.1689269107266949 :   2.7242079291862407 : False : False :  Reals
     C12H26 :  -4.300492592965126 : 0.09674373741815773 :   2.6690912034585947 : False : False :  Reals
     C13H26 : -5.0098534409819875 : -0.2918954117787228 :   2.3753473066026216 : False : False :  Reals
     C13H28 : -5.1395707941622035 : -0.3625336996948365 :   2.3224777739052733 : False : False :  Reals
   